In [ ]:
# %cd /content/drive/MyDrive/Files

/content/drive/MyDrive/Files


In [ ]:
!pip install pydub
!pip install soundfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import scipy.io.wavfile as wav
from pydub import AudioSegment
from sklearn.model_selection import train_test_split
from pydub.utils import make_chunks
import glob
import librosa
import soundfile
import pandas as pd
import numpy as np
import os
import joblib

In [ ]:
AVAILABLE_EMOTIONS = [
    "hu",
    "bp",
    "bu",
    "ti",
    "ch"
]
emotion2int = {e:i for i,e in enumerate(AVAILABLE_EMOTIONS)}

In [ ]:
directory = 'donateacry'
for folder in os.listdir(directory):
  new_fd = folder
  folder = os.path.join(directory,folder)
  for filename in os.listdir(folder):
    audio = AudioSegment.from_file(os.path.join(folder,filename))
    extract = audio[0:6000]
    chunk_length_ms = 1000
    chunks = make_chunks(extract, chunk_length_ms)
    filename = filename.split('.')[1][2:]
    for i, chunk in enumerate(chunks):
      chunk_name = "{}-{}.wav".format(filename,i)
      chunk.export(os.path.join("donateacry_processed", new_fd,chunk_name), format="wav")


In [ ]:
def extract_feature(file_name, **kwargs):
  
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")

    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))


    return result

def load_data(test_size=0.2, dir = "donateacry_processed"):
    X, y = [], []
    for folder in os.listdir(dir):
      folder = os.path.join(dir, folder)
      for file in glob.glob(folder+"/*.wav"):
          basename = os.path.basename(file)
          emotion = basename.split("-")[2]
          if emotion not in AVAILABLE_EMOTIONS:
              continue
          features = extract_feature(file, mfcc=True, chroma=True, mel=True)
          X.append(features)
          y.append(emotion2int[emotion])
    return X, y

In [ ]:
X, y = load_data(test_size=0.25)
X_train, X_test, y_train, y_test = train_test_split(np.array(X), y, test_size=0.2, random_state=7)

In [ ]:
print("[+] Number of training samples:", len(X_train))
print("[+] Number of testing samples:", len(X_test))

[+] Number of training samples: 148
[+] Number of testing samples: 38


In [ ]:
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.svm import SVC
from lightgbm import LGBMClassifier as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [ ]:
given = 0.92
for i in range(1000):
  xgmodel = XGBClassifier(
    max_depth = 3,
    colsample_bytree = 0.55,
    subsample = 0.873,
    alpha = 0.045
  )
  xgmodel.fit(X_train,y_train)
  accs = accuracy_score(xgmodel.predict(X_test),y_test)
  if accs>given:
    given = accs
    print(i, ":", accs)

7 : 0.9210526315789473
{0, 1, 2, 3, 4} {0, 1, 2, 3, 4}


In [ ]:
from xgboost import XGBClassifier

xgmodel = XGBClassifier( #max_depth = 3, colsample_bytree = 0.55, subsample = 0.873, alpha = 0.045
    max_depth = 3,
    colsample_bytree = 0.55,
    subsample = 0.873,
    alpha = 0.045
)
xgmodel.fit(X_train,y_train)
accuracy_score(xgmodel.predict(X_test),y_test) #92.10%

0.9210526315789473

In [ ]:
# joblib.dump(xgmodel, "xgbabycry.joblib")

['xgbabycry.joblib']